In [37]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import csv
from time import sleep
import ast
import pandas as pd
import urllib.request
import os

dir_driver="./chromedriver"
driver=webdriver.Chrome(dir_driver)
sleep(0.5)

keyword = '크루즈'
base_url="https://www.yna.co.kr/search/index?query={}".format(keyword)
driver.get(base_url)
object = bs(driver.page_source, 'html.parser')

# 제목 크롤링
subjects = object.find_all('span', {'class':'tt2'})
subjects = [subject.text for subject in subjects]

# 내용 크롤링
contents = object.find_all('span', {'class':'cts'})
contents = [content.text for content in contents]

# 이미지 크롤링
images = object.find('div', {'class':'cts_atclst'})
images = images.find_all('img')
images = ["https:" + image.get('src') for image in images]
for i in range(len(images)):
    urllib.request.urlretrieve(images[i], "./images/" + str(i) + ".jpg")

# 태그 크롤링
tags = list()
for i in range(1, 6):
    driver.find_element_by_xpath('//*[@id="article_list"]/div[2]/ul/li[{}]/a'.format(i)).click()
    sleep(0.1)
    object = bs(driver.page_source, 'html.parser')
    temp_tags = object.select('#articleWrap > div.content01.scroll-article-zone01 > div > div > div.keyword-zone > ul > li > a')
    temp_tags = [temp_tag.text for temp_tag in temp_tags]
    tags.append(temp_tags)
    driver.back()
    sleep(0.1)

data = {"subject" : subjects, "content":contents, "tags":tags, "images":images}
df = pd.DataFrame(data)
print(df.head(5))
driver.close()
#df.to_csv("test.csv", encoding = "utf-8-sig")

                                subject  \
0        中 출발 크루즈 6년간 제주 발길 끊어…올해도 '전무'   
1    [인천돋보기](25) 강릉엔 정동진·인천엔 정서진…해넘이 명소   
2         부산역 광장∼초량동 잇는 38ｍ 공중보행로 조성 추진   
3       갤러리아百, 명품 신발 선보인다…내년 루이비통·샤넬 오픈   
4  아시아-북미 잇는 '베링육교' 훨씬 늦은 3만5천700년 전 열려   

                                             content                     tags  \
0  ...어졌다.\n\n    2일 제주도가 확정한 2023년 국제크루즈 선석 배정 결...                   [#크루즈]   
1  ...uot;아라뱃길은 서해와 한강을 잇는 최초 뱃길로 우리 민족의 멋과 얼, 정서...    [#경인아라뱃길, #정서진, #해넘이]   
2  ...어 현재 실시설계 용역을 벌이고 있으며, 내년 7월 용역을 마무리하고 2024...           [#부산역, #공중보행로]   
3  ...으로 루이비통과 샤넬 슈즈를 선보일 예정이다. \n    디올 슈즈 매장은 디...               [#슈즈, #명품]   
4  ... \n    이는 약 11만 년 전 마지막 빙기가 시작되고 6만 년 이상 지난...  [#베링육교, #샌타크루즈 캘리포니아대학]   

                                              images  
0  https://img.yna.co.kr/photo/yna/YH/2017/03/16/...  
1  https://img.yna.co.kr/etc/inner/KR/2022/12/27/...  
2  https://img.yna.co.kr/photo/cms/2020/01/10/60/...  
3  https://img.yna.co.kr/etc/inner

In [38]:
!pip install psycopg2

In [71]:
import psycopg2

# Update connection string information
host = "localhost"
dbname = "postgres"
user = "postgres"
password = "postgres"
sslmode = "allow"

# Construct connection string
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string)
print("Connection established")

cursor = conn.cursor()

# Insert some data into the table
cursor.execute("SELECT news_no FROM news_contents;")
rows = cursor.fetchall()
news_no = max([row[0] for row in rows])

cursor.execute("SELECT seq FROM news_tag;")
rows = cursor.fetchall()
seq = max([row[0] for row in rows]) if len(rows)!=0 else 0

for i in range(len(subjects)):
    news_no += 1
    title = "'{}'".format(subjects[i].replace("'", "''", 10000))
    content = "'{}'".format(contents[i].replace("'", "''", 10000))
    mem_no = "'김가영'"
    crawl_dt = "'{}'".format(datetime.today().strftime("%Y-%m-%d"))
    crawl_tm = "'{}'".format(datetime.today().strftime("%H:%M:%S"))
    cursor.execute("INSERT INTO news_contents (news_no, title, content, mem_no, crawl_dt, crawl_tm) VALUES ({}, {}, {}, {}, {}, {});".format(news_no, title, content, mem_no, crawl_dt, crawl_tm))
    for tag in tags[i]:
        seq += 1
        tag = "'{}'".format()
        del_yn = "'N'"
        reg_mem_no = "'김가영'"
        reg_dt = "'{}'".format(datetime.today().strftime("%Y-%m-%d"))
        reg_tm = "'{}'".format(datetime.today().strftime("%H:%M:%S"))
        cursor.execute("INSERT INTO news_tag (news_no, seq, tag, del_yn, reg_mem_no, reg_dt, reg_tm) VALUES ({}, {}, {}, {}, {}, {}, {});".format(news_no, seq, tag, del_yn, reg_mem_no, reg_dt, reg_tm))
# Clean up
conn.commit()
cursor.close()
conn.close()
print('전부 삽입 완료!')

Connection established


UndefinedColumn: 오류:  "크루즈" 이름의 칼럼은 없습니다
LINE 1: ...l_yn, reg_mem_no, reg_dt, reg_tm) VALUES (13, 1, #크루즈, 'N...
                                                             ^
HINT:  아마 "news_tag.seq" 칼럼이나 "news_tag.tag" 칼럼을 참조하는 것 같습니다.


In [ ]:
list